In [1]:
import os

mycwd = os.getcwd()

os.chdir("..")

In [6]:
import tensorflow as tf
import gradio as gr
import requests
from notebooks.load_dataset.dataset import classes
import numpy as np

nm_model = tf.keras.models.load_model("../models/mn_model.keras")

resnet_model = tf.keras.models.load_model("../models/resnet_best.h5")

cifar10_labels = classes


def classify_image(inp, model_choice):
    try:
        print("Original image shape:", inp.shape)

        if model_choice == "MobileNetBased Model":
            inp = tf.image.resize(inp, (32, 32))

            print("Processed image for MobileNet model:", inp)
            model = nm_model
            labels = cifar10_labels
        elif model_choice == "ResNetBased Model":
            inp = tf.image.resize(inp, (32, 32))
            inp = tf.keras.applications.resnet.preprocess_input(inp)
            print("Processed image for ResNet model:", inp)
            model = resnet_model
            labels = cifar10_labels

        print("Resized image shape:", inp.shape)

        inp = tf.expand_dims(inp, axis=0)
        print("Input to the model:", inp)

        prediction = model.predict(inp).flatten()
        print("Predictions:", prediction)

        if model_choice == "MobileNetV2":
            top_indices = prediction.argsort()[-10:][::-1]
            confidences = {labels[i]: float(prediction[i]) for i in top_indices}
        else:
            confidences = {labels[i]: float(prediction[i]) for i in range(len(labels))}

        return confidences
    except Exception as e:
        return {"error": str(e)}


interface = gr.Interface(
    fn=classify_image,
    inputs=[gr.Image(type="numpy", image_mode="RGB", label="Input Image"), gr.Dropdown(["ResNetBased Model", "MobileNetBased Model"], value="ResNetBased Model", label="Model Choice")],
    outputs=gr.Label(num_top_classes=3, label="Predictions"),
)

interface.launch(debug=False, share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://df9dc6aef016e3142d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "e:\Work\Instruments\Anaconda\envs\ds_project\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 407, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "e:\Work\Instruments\Anaconda\envs\ds_project\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "e:\Work\Instruments\Anaconda\envs\ds_project\lib\site-packages\fastapi\applications.py", line 292, in __call__
    await super().__call__(scope, receive, send)
  File "e:\Work\Instruments\Anaconda\envs\ds_project\lib\site-packages\starlette\applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "e:\Work\Instruments\Anaconda\envs\ds_project\lib\site-packages\starlette\middleware\errors.py", line 184, in __call__
    raise exc
  File "e:\Work\Instruments\Anaconda\envs\ds_project\lib\site-packages\sta

Original image shape: (648, 836, 3)
Processed image for ResNet model: tf.Tensor(
[[[151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  ...
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]]

 [[151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  ...
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]]

 [[151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  ...
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]
  [151.061     138.22101   131.32     ]]

 ...

 [[ -1.095253   13.260063   37.585625 ]
  [ 11.8656845  29.924126   57.421562 ]
  [ 19.576622   48.31475    79.945    ]
  ...
  [-23.548378  -35.677437  -43.289375 ]
  [

In [4]:
import os
os.getcwd()

'e:\\Goit\\projects\\Data_Science_core\\final_data_project\\ds_project\\src'